In [1]:
import ultralytics
import cv2
import cv2.dnn
import numpy as np
import torch

In [4]:
model = ultralytics.YOLO('yolov8n.pt')
# export to onnx
model.export(format='onnx', opset=12)  # You can adjust the opset if needed



Ultralytics YOLOv8.2.98 🚀 Python-3.12.3 torch-2.4.1+cu121 CPU (AMD Ryzen 5 PRO 3400G with Radeon Vega Graphics)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'yolov8_supernew.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

ONNX: starting export with onnx 1.16.2 opset 12...
ONNX: export success ✅ 0.5s, saved as 'yolov8_supernew.onnx' (11.7 MB)

Export complete (1.0s)
Results saved to /home/taras-desktop/Documents/ucu/embeddedAI/project/model_training
Predict:         yolo predict task=detect model=yolov8_supernew.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8_supernew.onnx imgsz=640 data=/content//data.yaml  
Visualize:       https://netron.app


'yolov8_supernew.onnx'

In [36]:
img = 'image.png'
img = cv2.imread(img)
results = model(img)
for result in results:
    boxes = result.boxes  # Extract bounding boxes
    for box in boxes:
        print(box)
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get coordinates of the box
        confidence = box.conf[0]  # Get confidence score
        label = int(box.cls[0])  # Get the class label

        # Draw bounding box on the image
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Add label and confidence score
        cv2.putText(img, f'{model.names[label]} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
cv2.imwrite('output1.jpg', img)


0: 288x416 8 Trees, 159.4ms
Speed: 10.2ms preprocess, 159.4ms inference, 0.6ms postprocess per image at shape (1, 3, 288, 416)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9737])
data: tensor([[540.8509, 274.1178, 691.3217, 453.2866,   0.9737,   0.0000]])
id: None
is_track: False
orig_shape: (482, 723)
shape: torch.Size([1, 6])
xywh: tensor([[616.0863, 363.7022, 150.4708, 179.1688]])
xywhn: tensor([[0.8521, 0.7546, 0.2081, 0.3717]])
xyxy: tensor([[540.8509, 274.1178, 691.3217, 453.2866]])
xyxyn: tensor([[0.7481, 0.5687, 0.9562, 0.9404]])
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9711])
data: tensor([[ 42.5557, 271.5143, 214.1543, 461.3437,   0.9711,   0.0000]])
id: None
is_track: False
orig_shape: (482, 723)
shape: torch.Size([1, 6])
xywh: tensor([[128.3550, 366.4290, 171.5986, 189.8293]])
xywhn: tensor([[0.1775, 0.7602, 0.2373, 0.3938]])
xyxy: tensor([[ 42.5557, 271.5143, 214.1543, 461.34

True

In [132]:
!python3 inference.py --model yolov8n.onnx --img image.jpg --img_size 640

inference time:  0.19430756568908691
[1.2750852e+01 1.0473948e+01 2.8135237e+01 3.5534500e+01 7.1738388e-05]
draw time:  0.0001125335693359375
detection time:  0.2720773220062256
Output image saved as output.jpg and write time is:  1726940441.3801677


In [122]:
onnx_model = 'yolov8n.onnx'
input_image = 'image.png'
model = cv2.dnn.readNetFromONNX(onnx_model)

# Read the input image
original_image = cv2.imread(input_image)
[height, width, _] = original_image.shape
print(height, width)

# Prepare a square image for inference
length = max((height, width))
image = np.zeros((length, length, 3), np.uint8)
image[0:height, 0:width] = original_image

# Calculate scale factor
scale = length / 640

# Preprocess the image and prepare blob for model
blob = cv2.dnn.blobFromImage(image, scalefactor=1 / 255, size=(640, 640), swapRB=True)
model.setInput(blob)

# Perform inference
outputs = model.forward()
print(outputs.shape)    

# Prepare output array
outputs = np.array([cv2.transpose(outputs[0])])
rows = outputs.shape[1]

boxes = []
scores = []
class_ids = []

# Iterate through output to collect bounding boxes, confidence scores, and class IDs
for i in range(rows):
    classes_scores = outputs[0][i][4:]
    (minScore, maxScore, minClassLoc, (x, maxClassIndex)) = cv2.minMaxLoc(classes_scores)
    if maxScore >= 0.25:
        box = [
            outputs[0][i][0] - (0.5 * outputs[0][i][2]),
            outputs[0][i][1] - (0.5 * outputs[0][i][3]),
            outputs[0][i][2],
            outputs[0][i][3],
        ]
        boxes.append(box)
        scores.append(maxScore)
        class_ids.append(maxClassIndex)

# Apply NMS (Non-maximum suppression)
result_boxes = cv2.dnn.NMSBoxes(boxes, scores, 0.25, 0.45, 0.5)
print(result_boxes)
detections = []

# Draw the bounding boxes
for i in result_boxes:
    box = boxes[i]
    x = int(box[0] * length)
    y = int(box[1] * length)
    w = int(box[2] * length)
    h = int(box[3] * length)
    class_id = class_ids[i]
    score = scores[i]

    # Draw bounding box
    cv2.rectangle(original_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Add label and confidence score

cv2.imwrite('output2.jpg', original_image)
print(detections)


482 723
(1, 5, 8400)
[11 35 60 71  4 40 46 75]
[]


In [111]:
boxes = output[0][0].T  # Depending on how the output is structured, you might need further post-processing

# Extract and draw bounding boxes (assuming output[0] contains bounding boxes)
boxes = boxes[boxes[:, 4] > 0.9]  # Confidence threshold
print(len(boxes))
for box in boxes:
    x1, y1, x2, y2, confidence = box[:5]
    print(x1, y1, x2, y2, confidence)
    
    # Scale the coordinates back to original image size
    x1 = int(x1 * img.shape[1] / 640)
    y1 = int(y1 * img.shape[0] / 640)
    x2 = int(x2 * img.shape[1] / 640)
    y2 = int(y2 * img.shape[0] / 640)
    
    # Draw bounding box and label on the image
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, f'{confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

# Save the output image
cv2.imwrite('output_from_onnx.jpg', img)

29
271.44568 160.28482 75.45444 280.81732 0.9076861
390.99774 201.49124 143.35855 193.4961 0.9299027
271.01727 159.10193 78.1042 279.64343 0.91869575
391.71582 200.99384 144.02579 195.27289 0.937726
124.92833 205.88744 190.86374 189.12448 0.9232094
391.37473 201.23267 145.69696 196.30177 0.9362658
541.26013 204.48322 135.85938 191.45844 0.9062034
125.39397 207.0405 190.62463 189.1801 0.9342593
541.5865 203.6667 135.32642 192.55582 0.92351747
541.69073 203.97559 135.65787 192.58366 0.92040634
437.2132 469.05588 64.85556 251.33075 0.94387007
116.46841 475.4741 146.20439 247.42206 0.95187664
295.0421 482.11768 181.76395 235.08496 0.9406672
294.5409 481.31653 183.44617 233.98743 0.91053677
436.5722 467.8059 65.74353 252.18277 0.9571278
545.7831 480.35815 127.895325 233.35669 0.91712964
116.859375 474.53738 145.96951 244.13983 0.934174
295.3368 480.51147 180.86658 234.81946 0.9106795
295.29956 480.1389 182.37378 233.96698 0.93204004
294.88556 480.06958 182.852 233.8021 0.959304
436.6814 470

True

In [123]:
model5 = ultralytics.YOLO('yolov5.pt')
model5.export(format='onnx', opset=12)  # You can adjust the opset if needed

Ultralytics YOLOv8.2.98 🚀 Python-3.12.3 torch-2.4.1+cu121 CPU (AMD Ryzen 5 PRO 3400G with Radeon Vega Graphics)
YOLOv5n summary (fused): 193 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs

PyTorch: starting from 'yolov5.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 5, 3549) (5.0 MB)

ONNX: starting export with onnx 1.16.2 opset 12...
ONNX: export success ✅ 0.5s, saved as 'yolov5.onnx' (9.7 MB)

Export complete (0.9s)
Results saved to /home/taras-desktop/Documents/ucu/embeddedAI/project/model_training
Predict:         yolo predict task=detect model=yolov5.onnx imgsz=416  
Validate:        yolo val task=detect model=yolov5.onnx imgsz=416 data=/content/drive/MyDrive/Tree Decetion.v2-trees-v2.yolov5pytorch/data.yaml  
Visualize:       https://netron.app


'yolov5.onnx'